# 제3장 파이썬을 이용한 GIS 데이터 분석
___
## 3.2 __GIS 데이터의 처리와 표현__


## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumMyeongjo')
rc('axes', unicode_minus=False)

___

## 수원시 지구대(파출소) 위치 나타내기

### open API를 이용해 데이터 가져오기

* 경기데이터드림에서 수원시 지구대(파출소) 현황 데이터 가져오기

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, LineString, Point
import matplotlib.pyplot as plt
import requests as req
import json

* JSON 형식으로 결과 받기

In [ ]:
uri = 'https://openapi.gg.go.kr/Ptrldvsnsubpolcstus'
params = {
  'KEY' : '32f6008628b644da80a3be1ae80cdae7',
  'Type' : 'json',
  'pIndex' : 1,
  'pSize'  : 100,
  'SIGUN_NM' : '수원시'
}
resp = req.get(uri,params=params)
tmp = json.loads(resp.content)

In [ ]:
resp

In [ ]:
tmp

In [ ]:
tmp['Ptrldvsnsubpolcstus']

In [ ]:
tmp['Ptrldvsnsubpolcstus'][1]

In [ ]:
tmp['Ptrldvsnsubpolcstus'][1]['row']

In [ ]:
df1 = pd.json_normalize(tmp['Ptrldvsnsubpolcstus'][1]['row'])
df1.head()

In [ ]:
df1.columns

In [ ]:
df1.dtypes

* 필요한 정보만 추출

In [ ]:
sw_ps = df1[['SIGUN_CD','GOVOFC_NM','DIV_NM','REFINE_WGS84_LOGT','REFINE_WGS84_LAT']]

* 위도(LAT)와 경도(LOGT) 좌표를 이용해 공간 정보(점) 생성

In [ ]:
sw_ps['REFINE_WGS84_LOGT'] = sw_ps['REFINE_WGS84_LOGT'].astype(float)
sw_ps['REFINE_WGS84_LAT'] = sw_ps['REFINE_WGS84_LAT'].astype(float)

In [ ]:
sw_ps['geom'] = sw_ps.apply(lambda x : Point([x['REFINE_WGS84_LOGT'],x['REFINE_WGS84_LAT']]),axis=1)
sw_ps.head()

In [ ]:
sw_ps = gpd.GeoDataFrame(sw_ps,geometry='geom')
sw_ps.head()

In [ ]:
sw_ps.plot()
plt.show()

In [ ]:
sw_ps.plot(marker='^',color='black')
plt.show()

* 수원시 지도 데이터 가져오기

In [ ]:
emd = gpd.read_file('/content/BizDataAnalysis/DATA/emd.shp',encoding='CP949')
emd.head()

* 수원시 추출

In [ ]:
suwon = emd[emd['EMD_CD'].apply(lambda x : x.startswith('4111'))]

In [ ]:
suwon.head()

In [ ]:
suwon.plot()

In [ ]:
suwon.plot(column='EMD_KOR_NM',figsize=(8,8),cmap='coolwarm',alpha=0.7)

* 서울시 행정구역 지도 표현

In [ ]:
seoul = emd[emd['EMD_CD'].apply(lambda x : x.startswith('11'))]
seoul.plot()

* 좌표계 확인 및 지정

In [ ]:
suwon.crs

In [ ]:
suwon = suwon.to_crs({'init':'epsg:4326'})

In [ ]:
suwon.crs

In [ ]:
sw_ps.crs

In [ ]:
sw_ps.crs = {'init':'epsg:4326'}
sw_ps.crs

In [ ]:
fig, ax = plt.subplots(facecolor='white',figsize=(8,8))
ax.axis('off')
ax.set_title("수원시 지구대(파출소) 현황", fontsize=20)
suwon.plot(ax=ax,column='EMD_KOR_NM',cmap='coolwarm',alpha=0.7)
sw_ps.plot(ax=ax,marker='o',color='black',label='지구대(파출소)')
plt.legend()
plt.show()

* 구글 드라이브에 수원시 행정구역 지도 데이터와 지구대(파출소) 위치 데이터 저장

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
suwon.to_file('/content/drive/MyDrive/BizDataAnalysis/DATA/suwon.shp',driver='ESRI Shapefile',encoding='cp949')

In [ ]:
sw_ps.to_file('/content/drive/MyDrive/BizDataAnalysis/DATA/sw_ps.shp',driver='ESRI Shapefile',encoding='cp949')